<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/cosmo_xl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# COSMO 
# https://huggingface.co/allenai/cosmo-xl?text=Hey+my+name+is+Thomas%21+How+are+you%3F
# ref https://arxiv.org/pdf/2212.10465.pdf

In [2]:
!pip install transformers
!pip install sentencepiece
# had better restart runtime, maybe.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("allenai/cosmo-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/cosmo-xl")
model.to('cuda')

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/594 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [9]:
def set_input(narrative, instruction, dialogue_history):
    input_text = " <turn> ".join(dialogue_history)

    if instruction != "":
        input_text = instruction + " <sep> " + input_text

    if narrative != "":
        input_text = narrative + " <sep> " + input_text
    

    return input_text

def generate(narrative, instruction, dialogue_history):
    """
    narrative: the description of situation/context with the characters included (e.g., "David goes to an amusement park")
    instruction: the perspective/speaker instruction (e.g., "Imagine you are David and speak to his friend Sarah").
    dialogue history: the previous utterances in the dialogue in a list
    """

    input_text = set_input(narrative, instruction, dialogue_history) 

    inputs = tokenizer([input_text], return_tensors="pt")
    inputs = inputs.to('cuda')
    outputs = model.generate(inputs["input_ids"], max_new_tokens=128, temperature=1.0, top_p=.95, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

    return response

In [10]:
# Situation:  https://listentogenius.com/author.php/120
situation = "When Sue awoke from an hour's sleep the next morning she found Johnsy with dull, wide-open eyes staring at the drawn green shade."
instruction = "Johnsy is very ill and weak and the fever has left her mind morbid and full of strange fancies. "

dialogue = [
    "Try to sleep."
]

for i in range(1,30,1):
  response = generate(situation, instruction, dialogue)
  print(response)
  dialogue.append(response)

I can't. My mind won't stop racing.
You need to rest. You're not well enough to think clearly right now.
But what if I don't get better? What if this fever kills me?
Johnsy, you're going to be fine. I'm here with you and we'll get through this together.
I'm scared, Sue.
It's okay to be scared. Just try to focus on getting better and I'll be here with you every step of the way.
Alright.
I love you.
I love you too.
What did I tell you? About being scared?
Yes.
It's okay to be scared. We all are when we're faced with a scary situation. But you have to remember that I'm here with you and we'll get through this together, alright?
Okay, Sue.
Good girl. Now get some rest, okay? I'll check on you later.
All right. Love you.
Love you too.
Hey, Sue.
Hey, Johnsy. How are you feeling?
I'm feeling a little better today. The fever seems to be breaking a little bit and my mind is starting to focus more clearly.
That's great! You're finally starting to feel better. Just make sure you get some more res

In [11]:
!nvidia-smi

Sun Jan  8 05:46:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |  13578MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------